In [1]:
import math
import pandas
import numpy as np
from matplotlib import pyplot
pandas.set_option('display.max_columns', 100)
#pd.set_option('max_colwidth', 500)
pandas.set_option('display.max_rows', 500)

In [2]:
def get_cleaned_data(filename, exploratory=False, drop_outliers=False):
    if not exploratory:
        data_raw = pandas.read_csv(filename)
    else:
        data_raw = pandas.read_csv(filename).set_index('State').transpose().reset_index().rename(columns={'index': 'Date'})
        data_raw.columns.name = None
    outliers = set()
    daily_data = pandas.DataFrame()

    for column in data_raw:
        if column == 'Date':
            daily_data[column] = data_raw[column]
            continue
        
        daily_column = data_raw[column].diff().fillna(data_raw.iloc[0][column])
        daily_data[column] = daily_column
        sorted_column_data = np.array(daily_column.sort_values())

        Q3_index = math.ceil(75/100 * len(sorted_column_data))
        Q1_index = math.ceil(25/100 * len(sorted_column_data))
        alpha = 1.5
        IQR = sorted_column_data[Q3_index] - sorted_column_data[Q1_index]
        upper_threshold = sorted_column_data[Q3_index] + alpha * IQR
        lower_threshold = sorted_column_data[Q1_index] - alpha * IQR

        column_outliers = daily_column[(daily_column != 0) & ((daily_column < lower_threshold) | (daily_column > upper_threshold))].index
        outliers = outliers.union(column_outliers)
        #print ("\nColumn: %s \nDetected %s outlier rows \nIQR = %s \nlower thresold = %s \nupper thresold = %s" % (
        #column, len(column_outliers), IQR, lower_threshold, upper_threshold))
    
    if drop_outliers:
        data_raw.drop(outliers, inplace=True)
        daily_data.drop(outliers, inplace=True)

    return data_raw, daily_data

data, daily_data = get_cleaned_data('../data/US-all/US_confirmed.csv', exploratory=True, drop_outliers=False)
#daily_data[(daily_data['Date'] >= '2020-02-01') & (daily_data['Date'] <= '2020-06-30')][['Date', 'CA', 'NY', 'FL', 'TX']]


In [6]:
#def min_max_mean_daily_cases_month(daily_data, state):
#    state_data = daily_data[['Date', state]]
#    print(state_data.groupby(pandas.to_datetime(state_data['Date'])).sum()
#min_max_mean_daily_cases_month(daily_data, 'NY')
state_data = daily_data[(daily_data['Date'] >= '2020-01-01') & (daily_data['Date'] <= '2020-06-30')].copy()
state_data['Date'] = pandas.to_datetime(state_data['Date'])
monthly_cases_mean = state_data.groupby(pandas.Grouper(key='Date', freq='1M')).mean()
monthly_cases_mean.index = monthly_cases_mean.index.strftime('%-m %Y')
monthly_cases_mean['NY']
#print(state_data.groupby(pandas.to_datetime(state_data['Date'])).sum()

Date
1 2020       0.000000
2 2020       0.000000
3 2020    2401.516129
4 2020    7666.466667
5 2020    2141.870968
6 2020     755.233333
Name: NY, dtype: float64

In [5]:
monthly_cases_mean['NY'].idxmin()

'January'